<a href="https://colab.research.google.com/github/yuanjinren/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/YuanjinRen_LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
import sys
import os

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

In [ ]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [ ]:
#Shakespeare Data Parsed by Play
print(df_toc.shape)
df_toc.head()

(43, 4)


,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [ ]:
df_toc['text'][1]

'AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r\n\r\n  DUKE, living in exile\r\n  FREDERICK, his brother, and usurper of his dominions\r\n  AMIENS, lord attending on the banished Duke\r\n  JAQUES,   "      "       "  "     "      "\r\n  LE BEAU, a courtier attending upon Frederick\r\n  CHARLES, wrestler to Frederick\r\n  OLIVER, son of Sir Rowland de Boys\r\n  JAQUES,   "   "  "    "     "  "\r\n  ORLANDO,  "   "  "    "     "  "\r\n  ADAM,   servant to Oliver\r\n  DENNIS,     "     "   "\r\n  TOUCHSTONE, the court jester\r\n  SIR OLIVER MARTEXT, a vicar\r\n  CORIN,    shepherd\r\n  SILVIUS,     "\r\n  WILLIAM, a country fellow, in love with Audrey\r\n  A person representing HYMEN\r\n\r\n  ROSALIND, daughter to the banished Duke\r\n  CELIA, daughter to Frederick\r\n  PHEBE, a shepherdes\r\n  AUDREY, a country wench\r\n\r\n  Lords, Pages, Foresters, and Attendants\r\n\r\nSCENE: OLIVER\'S house; FREDERICK\'S court; and the Forest of Arden\r\n\r\nACT I. SCENE I. Orchard of OLIVER\'S house

In [ ]:
data = [t for t in df_toc['text']]

In [ ]:
len(data)

43

In [ ]:
# Encode Data as Chars
text = " ".join(data)
chars = list(set(text))

char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [ ]:
len(chars)

106

In [ ]:
char_int

{'\t': 95,
 '\n': 73,
 '\r': 46,
 ' ': 35,
 '!': 8,
 '"': 53,
 '$': 76,
 '%': 66,
 '&': 43,
 "'": 89,
 '(': 52,
 ')': 55,
 '*': 60,
 ',': 82,
 '-': 39,
 '.': 4,
 '/': 79,
 '0': 34,
 '1': 69,
 '2': 33,
 '3': 58,
 '4': 13,
 '5': 83,
 '6': 7,
 '7': 71,
 '8': 78,
 '9': 50,
 ':': 98,
 ';': 11,
 '?': 64,
 '@': 104,
 'A': 96,
 'B': 102,
 'C': 61,
 'D': 24,
 'E': 37,
 'F': 59,
 'G': 32,
 'H': 28,
 'I': 100,
 'J': 72,
 'K': 62,
 'L': 3,
 'M': 91,
 'N': 2,
 'O': 103,
 'P': 25,
 'Q': 85,
 'R': 75,
 'S': 16,
 'T': 87,
 'U': 54,
 'V': 38,
 'W': 56,
 'X': 0,
 'Y': 97,
 'Z': 45,
 '[': 57,
 '\\': 84,
 ']': 70,
 '_': 94,
 '`': 51,
 'a': 17,
 'b': 68,
 'c': 1,
 'd': 74,
 'e': 18,
 'f': 49,
 'g': 22,
 'h': 20,
 'i': 77,
 'j': 21,
 'k': 101,
 'l': 93,
 'm': 36,
 'n': 15,
 'o': 30,
 'p': 10,
 'q': 105,
 'r': 86,
 's': 26,
 't': 88,
 'u': 31,
 'v': 14,
 'w': 6,
 'x': 40,
 'y': 47,
 'z': 12,
 '|': 90,
 '}': 80,
 'Æ': 44,
 'É': 63,
 'à': 81,
 'â': 67,
 'æ': 29,
 'ç': 23,
 'è': 92,
 'é': 19,
 'ê': 27,
 'î': 42

In [ ]:
# Create the sequence data

maxlen = 40
step = 8
encoded = [char_int[c] for c in text]

sequences = []
next_char = []

for i in range(0, len(encoded)  - maxlen, step):
  sequences.append(encoded[i: i + maxlen])
  next_char.append(encoded[i + maxlen])

print('sequences: ', len(sequences))

sequences:  1900726


In [ ]:
sequences[0]

[35,
 96,
 16,
 35,
 97,
 103,
 54,
 35,
 3,
 100,
 62,
 37,
 35,
 100,
 87,
 46,
 73,
 46,
 73,
 46,
 73,
 24,
 75,
 96,
 91,
 96,
 87,
 100,
 16,
 35,
 25,
 37,
 75,
 16,
 103,
 2,
 96,
 37,
 4,
 46]

In [ ]:
for i in sequences[0]:
  print(int_char[i])

 
A
S
 
Y
O
U
 
L
I
K
E
 
I
T









D
R
A
M
A
T
I
S
 
P
E
R
S
O
N
A
E
.



In [ ]:
next_char[0], int_char[next_char[0]]

(73, '\n')

In [ ]:
# create x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [ ]:
x.shape

(1900726, 40, 106)

In [ ]:
y.shape

(1900726, 106)

In [ ]:
(maxlen, len(chars))

(40, 106)

In [ ]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        # sys.stdout.write(next_char)
        # sys.stdout.flush()
        print(next_char, end='')
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
# fit the model

model.fit(x, y,
          # batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
59398/59398 [==============================] - ETA: 0s - loss: 1.8202
----- Generating text after Epoch: 0
----- Generating with seed: "vining soul!
Methinks I see thee, now t"
vining soul!
Methinks I see thee, now to defullest her
Sark that in with you, the very man suppoll.
                  [Lo sHer' OUTISWel'T

  ALS. For I'll no nevingss.
    My lambre? A ampertant foptisoly leavy

  'TA Nayring ingood gute I am do that’s one was from parking treath,
Then do I gost potfort takes to the deave,
    She Rocentain 1 murtust, behall with Farsia
    Your d
59398/59398 [==============================] - 306s 5ms/step - loss: 1.8202
Epoch 2/10
59391/59398 [============================>.] - ETA: 0s - loss: 1.5434
----- Generating text after Epoch: 1
----- Generating with seed: " moreover, they have
      spoken untru"
 moreover, they have

    brears that antwick makes as the spow forts,
He extoge me to you
Biew. Why, I fashing in a prisons.

JULIAN.


PORTIA.


AJOX.



But, co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.



    And murder lates them; your drinds?                                                        Exeunt MURDER and A Strove,

    And that I ratee me not burry villain.
    Nothels mail; and thou 
59398/59398 [==============================] - 315s 5ms/step - loss: 1.3515


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN